In [3]:
!pip install pinecone-client openai python-dotenv pydparser

  Using cached pydparser-1.0.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached spacy-3.7.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached jsonschema-4.20.0-py3-none-any.whl.metadata (8.1 kB)
  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (11 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached pandas-2.1.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata (18 kB)
  Using cached docx2txt-0.8-py3-none-any.whl
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl.metadata (8.2 kB)
  Using cached pdfminer.six-20221105-py3-none-any.whl.metadata (4.0 kB)
  Using cached plac-1.4.2-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached datasets-2.16.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached pyarrow-15.0.2-cp311-cp311-macosx_10_15_x86_64.whl.metadata (3.0 kB)
  Using cached pyarrow_hot

In [5]:
from skillExtracter import extractKeywordFromLightSkillAPI
from util import getAccessToken
import json
token = json.loads(getAccessToken())

In [6]:
from pydparser import utils
import openai

import pandas as pd
import os
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [7]:
EMBEDDING_MODEL = "text-embedding-3-small"

In [8]:
from dotenv import load_dotenv

load_dotenv() 
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "None"))

In [9]:
def list_files(directory):
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

In [8]:
file_path = '../data/resumes/docx/Vivek.BSA.docx'
text = utils.extract_text(file_path, f".{file_path.split('/')[-1].split('.')[-1]}")


In [ ]:
from pydparser import ResumeParser
aaaa = ResumeParser(file_path).get_extracted_data().get("designation")[0]
print(aaaa)

In [10]:
def createEmbedding(text):
    response = client.embeddings.create(model=EMBEDDING_MODEL, input=text)
    for i, be in enumerate(response.data):
        assert i == be.index
    return [e.embedding for e in response.data]

In [ ]:
batch_embeddings = createEmbedding(text)

In [12]:
resumedf = pd.DataFrame({"values": batch_embeddings, "metadata": str({"resume_text": text})})
resumedf.reset_index(inplace=True)
resumedf.rename(columns={'index': 'id'}, inplace=True)

In [13]:
resumedf.columns

Index(['id', 'values', 'metadata'], dtype='object')

In [14]:
resumedf.to_csv("resume_embedding.csv")

In [15]:
batch_embeddings

[[-0.009777306579053402,
  0.024640686810016632,
  0.08250146359205246,
  0.012942718341946602,
  0.010861443355679512,
  -0.005243340041488409,
  -0.00995799619704485,
  0.02700972557067871,
  0.006635318044573069,
  0.026956187561154366,
  0.07468497008085251,
  -0.05509020388126373,
  -0.046979255974292755,
  0.011276359669864178,
  0.07671939581632614,
  0.046872179955244064,
  0.010988594964146614,
  -0.0019842376932501793,
  -0.04416853189468384,
  0.021040281280875206,
  0.047086331993341446,
  0.02965983748435974,
  0.006046404130756855,
  0.009161624126136303,
  0.021749654784798622,
  -0.06660079210996628,
  -0.0500308983027935,
  0.02166934870183468,
  0.028776466846466064,
  -0.06665432453155518,
  0.0392163023352623,
  -0.028803234919905663,
  0.0032791788689792156,
  0.03169426694512367,
  0.042027026414871216,
  0.029766913503408432,
  0.010754368267953396,
  0.0039149378426373005,
  0.030114907771348953,
  0.0038680925499647856,
  0.013946548104286194,
  -0.023007789626

In [13]:
directory = '../data/resumes/docx'

files = list_files(directory)
num = 1
resume_embed = []
for file_path in files:
    print(f"processing file - {file_path} , id - {num}")
    if file_path == "../data/resumes/docx/.DS_Store":
        print("in if")
        continue
    dic = {}
    _text = utils.extract_text(file_path, f".{file_path.split('/')[-1].split('.')[-1]}")
    batch_embeddings = createEmbedding(_text)
    skills = extractKeywordFromLightSkillAPI(_text, token) 
    dic["id"] = str(num)
    dic["values"] = batch_embeddings[0]
    designation = ResumeParser(file_path).get_extracted_data().get("designation")
    print(designation)
    if designation:
        dic["metadata"] = str({"designation": designation, "skills": skills})
    else:
        dic["metadata"] = str({"designation": "", "skills": skills})
    resume_embed.append(dic)
    num += 1
    

['Java Developer']
processing file - ../data/resumes/docx/Manohar Reddy.docx , id - 21
Spacy model is loading...
None
processing file - ../data/resumes/docx/Syed_Zia_Ashraf.docx , id - 22
Spacy model is loading...
None
processing file - ../data/resumes/docx/srinivas b.docx , id - 23
Spacy model is loading...
None
processing file - ../data/resumes/docx/Gururaja Murthy PMCPCSM.docx , id - 24
Spacy model is loading...
['Manager', 'Test Manager', 'Project Leader', 'Senior Test Engineer']
processing file - ../data/resumes/docx/Madhuri Pawar.docx , id - 25
Spacy model is loading...
['Senior Business Analyst']
processing file - ../data/resumes/docx/Sr. Business Analyst.docx , id - 26
Spacy model is loading...
['Sr. Business Analyst kr.reshma023@gmail.com']
processing file - ../data/resumes/docx/Naveen Kumar Bandela.docx , id - 27
Spacy model is loading...
None
processing file - ../data/resumes/docx/mohid_rj.docx , id - 28
Spacy model is loading...
['Java Developer']
processing file - ../data/

In [13]:
directory = '../data/resumes/docx'

files = list_files(directory)
num = 1
resume_embed_skill = []
for file_path in files:
    print(f"processing file - {file_path} , id - {num}")
    if file_path == "../data/resumes/docx/.DS_Store":
        print("in if")
        continue
    dic = {}
    _text = utils.extract_text(file_path, f".{file_path.split('/')[-1].split('.')[-1]}")
    skills = extractKeywordFromLightSkillAPI(_text, token) 
    if not skills:
        continue
    resume_skill_embeddings = createEmbedding(skills)
    dic["id"] = str(num)
    dic["values"] = resume_skill_embeddings[0]
    designation = ResumeParser(file_path).get_extracted_data().get("designation")
    print(designation)
    if designation:
        dic["metadata"] = str({"designation": designation, "skills": skills})
    else:
        dic["metadata"] = str({"designation": "", "skills": skills})
    resume_embed_skill.append(dic)
    num += 1
    

processing file - ../data/resumes/docx/Venkat_BA.docx , id - 1
Spacy model is loading...
None
processing file - ../data/resumes/docx/Shashank.docx , id - 2
Spacy model is loading...
['Business Analyst/Scrum Master']
processing file - ../data/resumes/docx/Anudeep N_Sr Java Developer.docx , id - 3
Spacy model is loading...
None
processing file - ../data/resumes/docx/ram nandyala.docx , id - 4
Spacy model is loading...
None
processing file - ../data/resumes/docx/Neha Mugghala.docx , id - 5
Spacy model is loading...
None
processing file - ../data/resumes/docx/chenna kesava.docx , id - 6
Spacy model is loading...
['Area Manager']
processing file - ../data/resumes/docx/Utthan Silawal12.docx , id - 7
Spacy model is loading...
['Sr. Business Analyst Gifferent and Brompton Ltd', 'Senior Business Analyst']
processing file - ../data/resumes/docx/Yohan BSA.docx , id - 8
Spacy model is loading...
['Sr. Business Systems Analyst']
processing file - ../data/resumes/docx/Madhu_BA_AW.DOCX , id - 9
proce

In [14]:
# resumedf = pd.DataFrame({"values": batch_embeddings, "metadata": str({"resume_text": text})})
resumedf = pd.DataFrame(resume_embed_skill)

In [24]:
resumedf

,id,values,metadata
0,1,"[-0.020351504907011986, 0.012719690799713135, ...","{'designation': '', 'skills': ['Data Store', '..."
1,2,"[-0.06397394090890884, 0.023600470274686813, 0...",{'designation': ['Business Analyst/Scrum Maste...
2,3,"[0.02591036446392536, 0.020131884142756462, 0....","{'designation': '', 'skills': ['C (Programming..."
3,4,"[0.02591036446392536, 0.020131884142756462, 0....","{'designation': '', 'skills': ['C (Programming..."
4,5,"[-0.007313738577067852, 0.04745469242334366, 0...","{'designation': '', 'skills': ['Oracle WebLogi..."
...,...,...,...
218,219,"[-0.01174512505531311, 0.012072647921741009, 0...","{'designation': ['Java Developer'], 'skills': ..."
219,220,"[-0.011879057623445988, 0.012870801612734795, ...","{'designation': '', 'skills': ['Debugging', 'A..."
220,221,"[-0.05594508722424507, 0.008369041606783867, 0...","{'designation': '', 'skills': ['Information Te..."
221,222,"[0.025919057428836823, 0.020163392648100853, 0...","{'designation': '', 'skills': ['C (Programming..."


In [19]:
resumedf.to_csv("../data/embeddings/resume_embedding.csv")

In [15]:
from pinecone import Pinecone, ServerlessSpec
_api_key = os.getenv("PINECONE_API_KEY")
print(_api_key)
pc = Pinecone(
    api_key="432efbce-a192-4eb1-bba7-6b46fb18f1b5"
)

432efbce-a192-4eb1-bba7-6b46fb18f1b5


/Users/aniketgiram/Desktop/Big-Data/JobFitAI/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [16]:
job_fit_index_name = 'job-fit'

In [119]:


# Check whether the index with the same name already exists - if so, delete it
if job_fit_index_name in pc.list_indexes():
    pc.delete_index(job_fit_index_name)

# Now do stuff
if job_fit_index_name not in pc.list_indexes().names():
    pc.create_index(
        name=job_fit_index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [41]:
index = pc.Index(job_fit_index_name)

In [122]:
resumedf.columns

Index(['id', 'values', 'metadata'], dtype='object')

In [18]:
def chunker(seq, size):
    """Yields successive chunks from seq."""
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos:pos + size]

In [19]:
def convert_data(chunk):
    """Converts a pandas DataFrame chunk to the format expected by Pinecone's upsert method."""
    data = []
    for _, row in chunk.iterrows():
        vector_id = str(row['id'])
        embedding = row['values']
        metadata = {"metadata": row.get('metadata', {})}
        data.append((vector_id, embedding, metadata))
    return data

In [45]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'description': {'vector_count': 7768},
                'resume_skills': {'vector_count': 389},
                'skills': {'vector_count': 7768}},
 'total_vector_count': 15925}

In [25]:

chunk_size = 100  # Define your chunk size
for chunk in chunker(resumedf, chunk_size):
    vectors_to_upsert = convert_data(chunk)
    print("vectorising chunk")
    index.upsert(
        vectors=vectors_to_upsert,
        namespace="resume_skills"
    )
    print("upserted chunk")

vectorising chunk
upserted chunk
vectorising chunk
upserted chunk
vectorising chunk
upserted chunk


In [51]:
df = pd.read_csv("../data/resumes/docx/Resume.csv")

In [54]:
df


,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [32]:
df = pd.read_csv("../predecting_job_titles.csv")

In [30]:
df = pd.read_csv("../data/resumes/UpdatedResumeDataSet.csv")

In [31]:
aadf = df.drop_duplicates(subset=['Resume'])

In [32]:
aadf

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
894,Testing,Computer Skills: â¢ Proficient in MS office (...
895,Testing,â Willingness to accept the challenges. â ...
896,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
897,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [27]:
def CreateEmbeddingWithVal(text, designation, num):
    embeddings = createEmbedding(text)
    dic["id"] = str(num)
    dic["values"] = embeddings[0]
    dic["metadata"] = str({"designation": designation, "skills": skills})
    return dic

In [33]:
num = 225
_skills_embedding = []

for index, row in aadf.iterrows():
    dic = {}
    if pd.isna(row.Category) or pd.isna(row.Resume):
        continue
    skills = extractKeywordFromLightSkillAPI(row.Resume, token) 
    
    embeddings = createEmbedding(skills)
    num += 1
    dic["id"] = str(num)
    dic["values"] = embeddings[0]
    dic["metadata"] = str({"designation": row.Category, "skills": skills})
    _skills_embedding.append(dic)

# Convert list of dictionaries to DataFrame
_skills_embeddings_df = pd.DataFrame(_skills_embedding)

In [ ]:
_skills_embedding = []
chunksize = 200
num = 225
for chunk in pd.read_csv('../data/resumes/UpdatedResumeDataSet.csv', chunksize=chunksize):
    print("processing batch")
    for row in chunk.itertuples(index=False):
        dic = {}
        if pd.isna(row.Category) or pd.isna(row.Resume):
            continue
        skills = extractKeywordFromLightSkillAPI(row.Resume, token) 
        print(skills)
        embeddings = createEmbedding(skills)
        num+=1
        dic["id"] = str(num)
        dic["values"] = embeddings[0]
        dic["metadata"] = str({"designation": row.Category, "skills": skills})
        _skills_embedding.append(dic)
    print("processed 200")
_skills_embeddings_df = pd.DataFrame(_skills_embedding)

In [38]:
_skills_embeddings_df

,id,values,metadata
0,226,"[-0.01007272768765688, 0.007961754687130451, 0...","{'designation': 'Data Science', 'skills': ['Ki..."
1,227,"[-0.02523578330874443, -0.033285126090049744, ...","{'designation': 'Data Science', 'skills': ['Ti..."
2,228,"[-0.020128333941102028, 0.022756729274988174, ...","{'designation': 'Data Science', 'skills': ['El..."
3,229,"[-0.00752217136323452, 0.014650698751211166, 0...","{'designation': 'Data Science', 'skills': ['In..."
4,230,"[-0.00017453814507462084, -0.01076367404311895...","{'designation': 'Data Science', 'skills': ['Da..."
...,...,...,...
161,387,"[-0.005178617779165506, 0.022664399817585945, ...","{'designation': 'Testing', 'skills': ['People ..."
162,388,"[0.025909531861543655, 0.02013123594224453, 0....","{'designation': 'Testing', 'skills': ['C (Prog..."
163,389,"[-0.03126473352313042, 0.017826635390520096, 0...","{'designation': 'Testing', 'skills': ['Functio..."
164,390,"[0.020757202059030533, -0.021839141845703125, ...","{'designation': 'Testing', 'skills': ['Printed..."


In [36]:
_skills_embeddings_df[_skills_embeddings_df['id'] == '261']

,id,values,metadata
35,261,"[0.010286211967468262, -0.01680522784590721, 0...","{'designation': 'Arts', 'skills': ['Honesty', ..."


In [43]:
chunk_size = 200  # Define your chunk size
for chunk in chunker(_skills_embeddings_df, chunk_size):
    vectors_to_upsert = convert_data(chunk)
    print("vectorising chunk")
    index.upsert(
        vectors=vectors_to_upsert,
        namespace="resume_skills"
    )
    print("upserted chunk")

vectorising chunk
upserted chunk


In [21]:
index.delete(namespace="resume_skills", delete_all=True)

{}